In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Unit Tests

In [2]:
nums = [1, 2, 4]

assert sum(nums) == 6, "Should be 6"

AssertionError: Should be 6

Python includes the **unittest** module which provides testing automatization, namely to write preparing and finishing code snippets for all tests, grouping tests, etc.

The basic concepts of the **unittest**:

**Test fixture** - the preparation is done to run the tests along with any necessary cleanup after the tests. This can include, for example, creating temporary databases or starting a server process.

**Test case** - minimal testing block. Checks answers on a given dataset. The unittest module provides the **TestCase** class which can be used to create new test cases.

**Test suite** - A group of test cases or a group of several test groups. It's used to combine various tests into one pipeline.

**Test runner** - a component that controls test execution and shows the results. 

In [3]:
#Example of testing the string methods

import unittest

class TestStringMethods(unittest.TestCase):
    # test method names should start with test
    def test_upper(self):
        self.assertEqual('foo'.upper(), 'FOO')

    def test_isupper(self):
        self.assertTrue('FOO'.isupper())
        self.assertFalse('Foo'.isupper())

    def test_split(self):
        s = 'hello world'
        self.assertEqual(s.split(), ['hello', 'world'])
        # Verify that s.split won't work for something that is not a string
        with self.assertRaises(TypeError):
            s.split(2)

if __name__ == '__main__':
    
    unittest.main()


E
ERROR: /Users/isklonin/Library/Jupyter/runtime/kernel-530d8c7e-9ca0-475a-a34d-6c1e00382460 (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute '/Users/isklonin/Library/Jupyter/runtime/kernel-530d8c7e-9ca0-475a-a34d-6c1e00382460'

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (errors=1)


SystemExit: True

/opt/homebrew/Caskroom/miniforge/base/envs/env3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
# Let's save last input from cell above

def dump_to(path):
    with open(path, 'w') as f:
        f.write(_i)  # _i is the "last executed Input" in iPython
        
dump_to('strings.py')

## Oops.. Let's make unittest friends with jupyter notebook:

Q: But what happened?

A: The reason is that unittest.main looks at sys.argv and first parameter is what started IPython or Jupyter, therefore the error about kernel connection file not being a valid attribute. Passing explicit list to unittest.main will prevent IPython and Jupyter look at sys.argv. Passing exit=False will prevent unittest.main to shutdown the kernell process

In [5]:
#Example of testing the string methods

class TestStringMethods(unittest.TestCase):
    # test method names should start with test
    def test_upper(self):
        self.assertEqual('foo'.upper(), 'FOO')

    def test_isupper(self):
        self.assertTrue('FOO'.isupper())
        self.assertFalse('Foo'.isupper())

    def test_split(self):
        s = 'hello world'
        self.assertEqual(s.split(), ['hello', 'world'])
        # Verify that s.split won't work for something that is not a string
        with self.assertRaises(TypeError):
            s.split(2)

if __name__ == '__main__':

    unittest.main(argv=['first-arg-is-ignored'], exit=False)
    

...
----------------------------------------------------------------------
Ran 3 tests in 0.002s

OK


## Command line interface

In [6]:
!python3 -m unittest strings                               # module
!python3 -m unittest strings.TestStringMethods             # class
!python3 -m unittest strings.TestStringMethods.test_split  # method

...
----------------------------------------------------------------------
Ran 3 tests in 0.000s

OK
...
----------------------------------------------------------------------
Ran 3 tests in 0.000s

OK
.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


In [7]:
#The -v flag provides the more detailed report:
!python3 -m unittest -v strings

test_isupper (strings.TestStringMethods) ... ok
test_split (strings.TestStringMethods) ... ok
test_upper (strings.TestStringMethods) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.000s

OK


## More flags:

-b (--buffer) - the program output on test failure will be shown instead of hidden as usual.

-c (--catch) - Ctrl + C, while a test is running, waits for the current test to complete and then reports the current results. Pressing Ctrl + C a second time throws a normal KeyboardInterrupt exception.

-f (--failfast) - exit after the first failed test.

--locals (starting with Python 3.5) - show local variables for failed tests.

## Test detection

unittest supports easy test detection. For compatibility with test detection, all test files must be modules or packages imported from the project's top-level directory.

Test detection is implemented in TestLoader.discover (), but can be used from the command line:

In [8]:
!mv strings.py test_strings.py  # rename the module to test....py to make it work
!python3 -m  unittest  discover

#-v (--verbose) - verbose output.
#-s (--start-directory) directory_name - test detection start directory (current by default).
#-p (--pattern) pattern - test file name template (test*.py by default).
#-t (--top-level-directory) directory_name - project top-level directory (default is start-directory).

...
----------------------------------------------------------------------
Ran 3 tests in 0.000s

OK


In [9]:
from unittest import TestLoader
help(TestLoader.discover)

Help on function discover in module unittest.loader:

discover(self, start_dir, pattern='test*.py', top_level_dir=None)
    Find and return all test modules from the specified start
    directory, recursing into subdirectories to find them and return all
    tests found within them. Only test files that match the pattern will
    be loaded. (Using shell style pattern matching.)
    
    All test modules must be importable from the top level of the project.
    If the start directory is not the top level directory then the top
    level directory must be specified separately.
    
    If a test package name (directory with '__init__.py') matches the
    pattern then the package will be checked for a 'load_tests' function. If
    this exists then it will be called with (loader, tests, pattern) unless
    the package has already had load_tests called from the same discovery
    invocation, in which case the package module object is not scanned for
    tests - this ensures that when a pack

## Test code organization

In [10]:
# Create some class that will be tested

class Widget():
    
    def __init__(self, name, x = 50, y = 50):
        self.name = name
        self.x = x
        self.y = y
        
    def size(self):
        return (self.x, self.y)
    
    def resize(self, x, y):
        self.x = x
        self.y = y



The basic test blocks are test cases - simple cases that must be checked for correctness.

The test case is created by inheriting from unittest.TestCase.

Testing code should be self-contained, that is, it should not depend in any way on other tests.

The simplest TestCase subclass can simply implement a test method (the method starting with test).

    

In [11]:
class DefaultWidgetSizeTestCase(unittest.TestCase):
    def test_default_widget_size(self):
        widget = Widget('The widget')
        self.assertEqual(widget.size(), (50, 50))

if __name__ == '__main__':

    unittest.main(argv=['',], defaultTest='DefaultWidgetSizeTestCase', exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


There can be many tests, and some of the configuration code can be repeated. Fortunately, we can define the setup code by implementing a **setUp()** method that will run _before_ each test.

We can also define a **tearDown()** method to run _after_ each test.

In [12]:
class SimpleWidgetTestCase(unittest.TestCase):
    def setUp(self):
        self.widget = Widget('The widget')

    def test_default_widget_size(self):
        self.assertEqual(self.widget.size(), (50, 50),
                         'incorrect default size')

    def test_widget_resize(self):
        self.widget.resize(100,150)
        self.assertEqual(self.widget.size(), (100, 150),
                         'wrong size after resize')
        
    def tearDown(self):
        pass
        
if __name__ == '__main__':

    unittest.main(argv=['',], defaultTest='SimpleWidgetTestCase', exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


It is possible to place all tests in the same file as the program itself (such as widgets.py), but placing the tests in a separate file (such as test_widget.py) has many advantages:

- The module with the test can be run autonomously from the command line.
- Test code can be easily separated from the program.
- Less temptation to change tests to match the program code for no apparent reason.
- The test code should be changed much less frequently than the program.
- Tested code can be more easily refactored.
- Tests for C modules should be in separate modules, so why not be consistent?
- If the testing strategy changes, there is no need to change the program code.

## Test skipping and expected fails

unittest supports skipping individual tests as well as test classes. In addition, it supports marking the test as "not working, but it should be."

The test is skipped using the **skip()** decorator or one of its conditional forms.

In [13]:
__version__ = (0, 9)
# __version__ = (1, 4)

platform = "ubuntu"
# platform = "windows"


class MyTestCase(unittest.TestCase):

    @unittest.skip("demonstrating skipping")
    def test_nothing(self):
        self.fail("shouldn't happen")

    @unittest.skipIf(__version__ < (1, 3),
                     "not supported in this library version")
    def test_format(self):
        # Tests that work for only a certain version of the library.
        pass

    @unittest.skipUnless(platform.startswith("win"), "requires Windows")
    def test_windows_support(self):
        # windows specific testing code
        pass

if __name__ == '__main__':

    unittest.main(argv=['', '-v'], defaultTest='MyTestCase', exit=False)

test_format (__main__.MyTestCase) ... skipped 'not supported in this library version'
test_nothing (__main__.MyTestCase) ... skipped 'demonstrating skipping'
test_windows_support (__main__.MyTestCase) ... skipped 'requires Windows'

----------------------------------------------------------------------
Ran 3 tests in 0.002s

OK (skipped=3)


#### Classes may be skipped too:

In [14]:
@unittest.skip("showing class skipping")
class MySkippedTestCase(unittest.TestCase):
    def test_not_run(self):
        pass
    
if __name__ == '__main__':

    unittest.main(argv=['', '-v'], defaultTest='MySkippedTestCase', exit=False)

test_not_run (__main__.MySkippedTestCase) ... skipped 'showing class skipping'

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK (skipped=1)


#### expectedFailure() is used for expected fails:

In [15]:
class ExpectedFailureTestCase(unittest.TestCase):
    @unittest.expectedFailure
    def test_fail(self):
        self.assertEqual(1, 0, "broken")
#         self.assertEqual(0, 0, "broken")

if __name__ == '__main__':

    unittest.main(argv=['', '-v'], defaultTest='ExpectedFailureTestCase', exit=False)        

test_fail (__main__.ExpectedFailureTestCase) ... expected failure

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK (expected failures=1)


#### It's very easy to make your own decorator. For example, the following decorator skips the test if the passed object does not have the specified attribute.


SetUp() and tearDown() are not triggered for missed tests. SetUpClass() and tearDownClass() are not triggered for missing classes. For missing modules setUpModule() and tearDownModule() are not triggered.

In [16]:
obj1 = [1, 2, 3]


def skipUnlessHasattr(obj, attr):
    if hasattr(obj, attr):
        return lambda func: func
    return unittest.skip("{!r} doesn't have {!r}".format(obj, attr))


class YetAnotherTestCase(unittest.TestCase):
    @skipUnlessHasattr(obj1, 'add')  # append
    def test_fail(self):
        pass

if __name__ == '__main__':

    unittest.main(argv=['', '-v'], defaultTest='YetAnotherTestCase', exit=False)        

test_fail (__main__.YetAnotherTestCase) ... skipped "[1, 2, 3] doesn't have 'add'"

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK (skipped=1)


#### Hey, what's up with setUpClass() and setUpModule() ??

In [17]:
import unittest

class Test(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        cls._connection = createExpensiveConnectionObject()

    @classmethod
    def tearDownClass(cls):
        cls._connection.destroy()
        

#These should be implemented as functions:

def setUpModule():
    createConnection()

def tearDownModule():
    closeConnection()
    

# this code isnt executable (for show only), so delete objects
del Test, setUpModule, tearDownModule

#### Distinguishing test iterations using subtests

When some tests have only minor differences, such as some parameters, unittest allows you to distinguish them within a single test method using the **subTest()** context manager.

In [18]:
class NumbersTest(unittest.TestCase):

    def test_even(self):
        """
        Test that numbers between 0 and 3 are all even.
        """
        for i in range(0, 4):
#             self.assertEqual(i % 2, 0)  # or
            with self.subTest(i=i):
                self.assertEqual(i % 2, 0)
                
unittest.main(argv=['', '-v'], defaultTest='NumbersTest', exit=False)           

test_even (__main__.NumbersTest)
Test that numbers between 0 and 3 are all even. ... 
FAIL: test_even (__main__.NumbersTest) (i=1)
Test that numbers between 0 and 3 are all even.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/v8/2cskhqjj0_g0zp3824glf1tm0000gn/T/ipykernel_54888/2430741492.py", line 10, in test_even
    self.assertEqual(i % 2, 0)
AssertionError: 1 != 0

FAIL: test_even (__main__.NumbersTest) (i=3)
Test that numbers between 0 and 3 are all even.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/v8/2cskhqjj0_g0zp3824glf1tm0000gn/T/ipykernel_54888/2430741492.py", line 10, in test_even
    self.assertEqual(i % 2, 0)
AssertionError: 1 != 0

----------------------------------------------------------------------
Ran 1 test in 0.003s

FAILED (failures=2)


#### Success checks

The unittest module provides many functions for a wide variety of tests:

```
assertEqual(a, b) — a == b

assertNotEqual(a, b) — a != b

assertTrue(x) — bool(x) is True

assertFalse(x) — bool(x) is False

assertIs(a, b) — a is b

assertIsNot(a, b) — a is not b

assertIsNone(x) — x is None

assertIsNotNone(x) — x is not None

assertIn(a, b) — a in b

assertNotIn(a, b) — a not in b

assertIsInstance(a, b) — isinstance(a, b)

assertNotIsInstance(a, b) — not isinstance(a, b)

assertRaises(exc, fun, *args, **kwds) — fun(*args, **kwds) raises exc exception

assertRaisesRegex(exc, r, fun, *args, **kwds) — fun(*args, **kwds) throws exc exception and message matches regex r

assertWarns(warn, fun, *args, **kwds) — fun(*args, **kwds) raises warning

assertWarnsRegex(warn, r, fun, *args, **kwds) — fun(*args, **kwds) raises warning and message matches regex r

assertAlmostEqual(a, b) — round(a-b, 7) == 0

assertNotAlmostEqual(a, b) — round(a-b, 7) != 0

assertGreater(a, b) — a > b

assertGreaterEqual(a, b) — a >= b

assertLess(a, b) — a < b

assertLessEqual(a, b) — a <= b

assertRegex(s, r) — r.search(s)

assertNotRegex(s, r) — not r.search(s)

assertCountEqual(a, b) — a & b contain the same elements in the same quantities, but the order is not important
```

#### To customize the execution of given tests:

In [19]:
def MySuite():
    suite = unittest.TestSuite()
    suite.addTest(SimpleWidgetTestCase('test_default_widget_size'))
    suite.addTest(SimpleWidgetTestCase('test_widget_resize'))
    suite.addTest(NumbersTest('test_even'))
    suite.addTest(YetAnotherTestCase('test_fail'))
    return suite

if __name__ == '__main__':
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(MySuite())

test_default_widget_size (__main__.SimpleWidgetTestCase) ... ok
test_widget_resize (__main__.SimpleWidgetTestCase) ... ok
test_even (__main__.NumbersTest)
Test that numbers between 0 and 3 are all even. ... test_fail (__main__.YetAnotherTestCase) ... skipped "[1, 2, 3] doesn't have 'add'"

FAIL: test_even (__main__.NumbersTest) (i=1)
Test that numbers between 0 and 3 are all even.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/v8/2cskhqjj0_g0zp3824glf1tm0000gn/T/ipykernel_54888/2430741492.py", line 10, in test_even
    self.assertEqual(i % 2, 0)
AssertionError: 1 != 0

FAIL: test_even (__main__.NumbersTest) (i=3)
Test that numbers between 0 and 3 are all even.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/v8/2cskhqjj0_g0zp3824glf1tm0000gn/T/ipykernel_54888/2430741492.py", line 10, in test_even
    self.assertEqual(i % 2, 0)
Ass

<unittest.runner.TextTestResult run=4 errors=0 failures=2>